In [1]:
import sys
sys.path.append('/Users/jdli/Project/Korg.jl/Jorg/src/')



In [2]:
from jorg.statmech import (
    # Core chemical equilibrium
    chemical_equilibrium,
    # Species representation
    Species, Formula,
    
    # Data creation functions
    create_default_partition_functions,
    create_default_ionization_energies,
    create_default_log_equilibrium_constants,
    
    # Low-level functions
    saha_ion_weights,
    translational_U,
    get_log_nK
)

In [3]:
from jorg.statmech import Species, Formula

# Atomic species using different notation styles
h_neutral = Species.from_string("H I")        # Roman numeral notation
h_ion = Species.from_string("H II")           # Singly ionized hydrogen
h_plus = Species.from_string("H+")            # Alternative notation
h_minus = Species.from_string("H-")           # Hydrogen anion

# Iron species
fe_neutral = Species.from_string("Fe I")      # Neutral iron
fe_ion = Species.from_string("Fe II")         # Singly ionized iron

# Molecular species
co = Species.from_string("CO")                # Carbon monoxide
h2o = Species.from_string("H2O")              # Water
oh = Species.from_string("OH")                # Hydroxyl radical

# MOOG-style numeric codes
co_moog = Species.from_string("0608")         # CO (C=6, O=8)

print(f"Hydrogen neutral: {h_neutral}")
print(f"Iron ion: {fe_ion}")
print(f"Carbon monoxide: {co}")
print(f"Water: {h2o}")

Hydrogen neutral: H I
Iron ion: Fe II
Carbon monoxide: CO
Water: H2O


In [4]:
# Examine species properties
species = Species.from_string("Fe II")

print(f"Species: {species}")
print(f"Is atom: {species.is_atom}")
print(f"Is molecule: {species.is_molecule}")  
print(f"Is neutral: {species.is_neutral}")
print(f"Is ion: {species.is_ion}")
print(f"Charge: {species.charge}")
print(f"Mass (AMU): {species.mass}")
print(f"Number of atoms: {species.n_atoms}")
print(f"Atomic composition: {species.get_atoms()}")

# For molecules
h2o = Species.from_string("H2O")
print(f"\nWater molecule:")
print(f"  Formula: {h2o.formula}")
print(f"  Mass: {h2o.mass:.3f} AMU")
print(f"  Atoms: {h2o.get_atoms()}")  # (1, 1, 8) - two H, one O

Species: Fe II
Is atom: True
Is molecule: False
Is neutral: False
Is ion: True
Charge: 1
Mass (AMU): 55.85
Number of atoms: 1
Atomic composition: (26,)

Water molecule:
  Formula: H2O
  Mass: 18.016 AMU
  Atoms: (1, 1, 8)


In [5]:
# Examine species properties
species = Species.from_string("Fe II")

print(f"Species: {species}")
print(f"Is atom: {species.is_atom}")
print(f"Is molecule: {species.is_molecule}")  
print(f"Is neutral: {species.is_neutral}")
print(f"Is ion: {species.is_ion}")
print(f"Charge: {species.charge}")
print(f"Mass (AMU): {species.mass}")
print(f"Number of atoms: {species.n_atoms}")
print(f"Atomic composition: {species.get_atoms()}")

# For molecules
h2o = Species.from_string("H2O")
print(f"\nWater molecule:")
print(f"  Formula: {h2o.formula}")
print(f"  Mass: {h2o.mass:.3f} AMU")
print(f"  Atoms: {h2o.get_atoms()}")  # (1, 1, 8) - two H, one O

Species: Fe II
Is atom: True
Is molecule: False
Is neutral: False
Is ion: True
Charge: 1
Mass (AMU): 55.85
Number of atoms: 1
Atomic composition: (26,)

Water molecule:
  Formula: H2O
  Mass: 18.016 AMU
  Atoms: (1, 1, 8)


In [6]:
# Create formulas directly
h_formula = Formula.from_atomic_number(1)     # Hydrogen
co_formula = Formula.from_string("CO")        # Carbon monoxide
h2o_formula = Formula.from_string("H2O")      # Water

# Use formulas to create species
h_neutral = Species(h_formula, 0)             # H I
h_ion = Species(h_formula, 1)                 # H II

print(f"Hydrogen formula: {h_formula}")
print(f"CO formula: {co_formula}")
print(f"H2O atoms: {h2o_formula.get_atoms()}")

Hydrogen formula: H
CO formula: CO
H2O atoms: (1, 1, 8)


In [7]:
from jorg.statmech import (
    create_default_partition_functions,
    create_default_ionization_energies,
    create_default_log_equilibrium_constants
)

# Load default data sets
partition_funcs = create_default_partition_functions()
ionization_energies = create_default_ionization_energies()
log_equilibrium_constants = create_default_log_equilibrium_constants()

print(f"Loaded partition functions for {len(partition_funcs)} species")
print(f"Loaded ionization energies for {len(ionization_energies)} elements")
print(f"Loaded equilibrium constants for {len(log_equilibrium_constants)} molecules")

Loaded partition functions for 276 species
Loaded ionization energies for 92 elements
Loaded equilibrium constants for 9 molecules


In [8]:
import jax.numpy as jnp

# Check ionization energies (in eV)
print("Ionization energies (eV):")
for element in [1, 2, 6, 8, 26]:  # H, He, C, O, Fe
    if element in ionization_energies:
        chi_I, chi_II, chi_III = ionization_energies[element]
        print(f"  Z={element}: χI={chi_I:.3f}, χII={chi_II:.3f}, χIII={chi_III:.3f}")

# Check partition functions
h_i = Species.from_string("H I")
fe_i = Species.from_string("Fe I")
fe_ii = Species.from_string("Fe II")

T = 5778.0  # Solar temperature
log_T = jnp.log(T)

print(f"\nPartition functions at T={T}K:")
print(f"  H I: {partition_funcs[h_i](log_T):.2f}")
print(f"  Fe I: {partition_funcs[fe_i](log_T):.2f}")
print(f"  Fe II: {partition_funcs[fe_ii](log_T):.2f}")

# Check molecular equilibrium constants
co = Species.from_string("CO")
oh = Species.from_string("OH")

print(f"\nLog equilibrium constants at T={T}K:")
if co in log_equilibrium_constants:
    print(f"  CO: {log_equilibrium_constants[co](log_T):.2f}")
if oh in log_equilibrium_constants:
    print(f"  OH: {log_equilibrium_constants[oh](log_T):.2f}")

Ionization energies (eV):
  Z=1: χI=13.598, χII=-1.000, χIII=-1.000
  Z=2: χI=24.587, χII=54.418, χIII=-1.000
  Z=6: χI=11.260, χII=24.385, χIII=47.888
  Z=8: χI=13.618, χII=35.121, χIII=54.936
  Z=26: χI=7.902, χII=16.199, χIII=30.651

Partition functions at T=5778.0K:
  H I: 2.00
  Fe I: 25.00
  Fe II: 30.00

Log equilibrium constants at T=5778.0K:
  CO: -4.65
  OH: -2.74


In [9]:
import numpy as np
from jorg.statmech import chemical_equilibrium
from jorg.abundances import format_A_X

# Define stellar atmosphere conditions
T = 5778.0      # Temperature (K) - solar photosphere
nt = 1e15       # Total number density (cm⁻³)
model_atm_ne = 1e12  # Initial electron density guess (cm⁻³)

# Get solar abundances and convert to absolute scale
abundances = format_A_X()  # Returns log abundances relative to H=12
absolute_abundances = {}

for Z, log_abundance in abundances.items():
    absolute_abundances[Z] = 10**(log_abundance - 12.0)

# Normalize abundances so sum(N_X/N_total) = 1
total_abundance = sum(absolute_abundances.values())
for Z in absolute_abundances:
    absolute_abundances[Z] /= total_abundance

print(f"Solar abundances loaded for {len(absolute_abundances)} elements")
print(f"Hydrogen abundance: {absolute_abundances[1]:.3e}")
print(f"Helium abundance: {absolute_abundances[2]:.3e}")
print(f"Iron abundance: {absolute_abundances[26]:.3e}")

Solar abundances loaded for 30 elements
Hydrogen abundance: 9.207e-01
Helium abundance: 7.836e-02
Iron abundance: 2.911e-05


In [10]:
# Calculate chemical equilibrium
ne, number_densities = chemical_equilibrium(
    T, nt, model_atm_ne, absolute_abundances,
    ionization_energies, partition_funcs, log_equilibrium_constants
)

print(f"\nChemical equilibrium results:")
print(f"  Temperature: {T} K")
print(f"  Total density: {nt:.2e} cm⁻³")
print(f"  Electron density: {ne:.2e} cm⁻³")
print(f"  Species calculated: {len(number_densities)}")


Chemical equilibrium results:
  Temperature: 5778.0 K
  Total density: 1.00e+15 cm⁻³
  Electron density: 1.00e+13 cm⁻³
  Species calculated: 99


/Users/jdli/anaconda3/envs/va/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/jdli/Project/Korg.jl/Jorg/src/jorg/statmech/chemical_equilibrium.py:156: UserWarning: Calculated ne (1.00e+13) differs from model (1.00e+12) by 900.0%
  warnings.warn(f"Calculated ne ({ne_calc:.2e}) differs from model ({model_atm_ne:.2e}) by {rel_diff:.1%}")


In [11]:
key_species = [
    ("H I", "H II", "H-"),           # Hydrogen
    ("He I", "He II"),               # Helium  
    ("C I", "C II"),                 # Carbon
    ("O I", "O II"),                 # Oxygen
    ("Fe I", "Fe II"),               # Iron
]

print("\nAtomic species number densities:")
for species_group in key_species:
    print(f"  {species_group[0][:-2]}:")  # Element name
    for species_name in species_group:
        species = Species.from_string(species_name)
        density = number_densities.get(species, 0.0)
        print(f"    {species_name}: {density:.3e} cm⁻³")

# Molecular species
molecular_species = ["CO", "OH", "H2", "H2O"]
print("\nMolecular species number densities:")
for mol_name in molecular_species:
    try:
        species = Species.from_string(mol_name)
        density = number_densities.get(species, 0.0)
        print(f"  {mol_name}: {density:.3e} cm⁻³")
    except:
        print(f"  {mol_name}: not found")


Atomic species number densities:
  H:
    H I: 9.101e+14 cm⁻³
    H II: 1.329e+11 cm⁻³
    H-: 0.000e+00 cm⁻³
  He:
    He I: 7.758e+13 cm⁻³
    He II: 1.178e+01 cm⁻³
  C:
    C I: 2.148e+11 cm⁻³
    C II: 3.053e+09 cm⁻³
  O:
    O I: 4.459e+11 cm⁻³
    O II: 5.463e+07 cm⁻³
  Fe:
    Fe I: 8.821e+07 cm⁻³
    Fe II: 2.873e+09 cm⁻³

Molecular species number densities:
  CO: 1.000e-10 cm⁻³
  OH: 1.000e-10 cm⁻³
  H2: 1.000e-10 cm⁻³
  H2O: 1.000e-10 cm⁻³
